# COURSE: Master Python for scientific programming by solving projects
## PROJECT: Animate data
#### TEACHER: Mike X Cohen, sincxpress.com
##### COURSE URL: udemy.com/course/maspy_x/?couponCode=202201

In [ ]:
# import modules
import plotly.graph_objects as go

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# set animation defaults
from matplotlib import rc
rc('animation', html='jshtml')

# Wavey wavelets in plotly

In [ ]:
def createComplexWavelet(time,freq,fwhm,phs=0):
  sinepart = np.exp( 1j*(2*np.pi*freq*time + phs) )
  gauspart = np.exp( (-4*np.log(2)*time**2)/(fwhm**2) )
  return sinepart*gauspart

In [ ]:
# parameters
freq  = 5 # Hz
fwhm  = .5
srate = 500 # Hz
time  = np.arange(-2*srate,2*srate)/srate
npnts = len(time)


# create a complex Morlet wavelet
wavelet = createComplexWavelet(time,5,1)

# vanilla visualization
plt.plot(time,np.real(wavelet),label='Real part')
plt.plot(time,np.imag(wavelet),label='Imaginary part')
plt.plot(time,np.abs(wavelet),'k',label='Magnitude')

plt.xlabel('Time (s)')
plt.legend(frameon=False)
plt.show()

In [ ]:
fig = go.Figure(
    
    # 'data' sets the initial graph data
    data = [ go.Scatter(x=time, y=np.real(createComplexWavelet(time,freq,fwhm)),mode='lines') ],
    
    # layout defines the figure layout
    layout = go.Layout(updatemenus = [dict(type  = 'buttons',
                                         buttons = [ {'label':'Play', 
                                                      'method':'animate', 'args':[None]}  ])]   ),
    
    # frames is a list of movie frames
    frames = [go.Frame(data=[go.Scatter(x=time, y=np.real(createComplexWavelet(time,freq,fwhm,np.pi/6)))]),
              go.Frame(data=[go.Scatter(x=time, y=np.real(createComplexWavelet(time,freq,fwhm,np.pi/4)))]),
              go.Frame(data=[go.Scatter(x=time, y=np.real(createComplexWavelet(time,freq,fwhm,np.pi/3)))]) ]
)

fig.show()

In [ ]:
# make figure
gofigure = {
    'data': [ go.Scatter(x=time, y=np.real(createComplexWavelet(time,freq,fwhm)),name='Real part'),
              go.Scatter(x=time, y=np.imag(createComplexWavelet(time,freq,fwhm)),name='Imag part')],
    
    'layout': go.Layout(updatemenus = [dict(type='buttons',
                        buttons     = [ {'label':'Play', 'method':'animate',
                                         'args':[None]}  ])]    ,
                        title = 'Complex Morlet wavelet'
                        ),
    
    'frames': []
}


# create frames
phases = np.linspace(0,2*np.pi,10)
for phi in phases:
  frames  = {'data':[]}
  tmpdct1 = {'x':time, 'y':np.real(createComplexWavelet(time,freq,fwhm,phi))}
  tmpdct2 = {'x':time, 'y':np.imag(createComplexWavelet(time,freq,fwhm,phi))}
  frames['data'].append(tmpdct1)
  frames['data'].append(tmpdct2)
  gofigure['frames'].append(frames)

fig = go.Figure(gofigure)
fig.show()


In [ ]:
# an example of the ludicrous complexity of the go structure
gofigure['frames'][4]['data'][0]['x']

# Wavey wavelets in matplotlib

In [ ]:
# function to draw the plots
def aframe(phs):

  # create the wavelet
  wavelet = createComplexWavelet(time,freq,fwhm,phs)

  # update the figure
  plth1.set_ydata(np.real(wavelet))
  plth2.set_ydata(np.imag(wavelet))
  return (plth1,plth2)

In [ ]:
# setup figure
fig,ax = plt.subplots(1,figsize=(12,6))

plth1, = ax.plot(time,np.zeros(npnts))
plth2, = ax.plot(time,np.zeros(npnts))
ax.set_ylim([-1,1])

# note: In the video, I forgot to re-define phases with higher resolution.
#       Note also cutting off the final phase values to avoid having the
#       the plot start and end with the same phase value. That makes it smoother ;)
phases = np.linspace(0,2*np.pi-2*np.pi/10,10)
ani = animation.FuncAnimation(fig, aframe, phases, interval=50, repeat=True)

In [ ]:
# now run the animation
ani

# Mobius transform in matplotlib

In [ ]:
# data parameters and initializations
lim = 40

a = np.linspace(-lim,lim,80)
b = np.linspace(-lim,lim,75)

mobtrans = np.zeros((len(a),len(b)),dtype=complex)
ts = np.linspace(.2,2,90)



# function to draw the plots
def aframe(t):

  # create the mobius transform
  for i,aa in enumerate(a):
    for j,bb in enumerate(b):

      # a complex number
      z = np.complex(aa,bb)

      # the formula
      num = (t-1) + (t+1)*z
      den = (t+1) + (t-1)*z
      mobtrans[i,j] = num / den

  # update the figure
  imh[0].set_data(np.real(mobtrans))
  imh[1].set_data(np.imag(mobtrans))
  imh[2].set_data(np.abs(mobtrans))
  
  return imh

In [ ]:
# setup figure
fig,axs = plt.subplots(1,3,figsize=(12,8))

imh = [0]*3
imh[0] = axs[0].imshow(np.zeros((len(a),len(b))),vmin=-50,vmax=50)
imh[1] = axs[1].imshow(np.zeros((len(a),len(b))),vmin=-50,vmax=50)
imh[2] = axs[2].imshow(np.zeros((len(a),len(b))),vmin=-50,vmax=50)

for i in range(3):
  axs[i].tick_params(labelbottom=False,labelleft=False)

axs[0].set_title('Real')
axs[1].set_title('Imag')
axs[2].set_title('Magnitude')

ani = animation.FuncAnimation(fig, aframe, ts, interval=50, repeat=True )
ani

In [ ]:
# save as a gif
writergif = animation.PillowWriter(fps=30) 
ani.save('MobiusTransformAnimation.gif', writer=writergif)

# Bonus: The wandering prime

In [ ]:
# inspiration: 
# https://www.reddit.com/r/dataisbeautiful/comments/jdmxby/oc_prime_numbers_whenever_n_was_a_prime_number/

In [ ]:
from sympy import isprime

# maximum possible number to test for primes
n = 100000

# using list comprehension
ps = [ i for i in range(n) if isprime(i) ]

In [ ]:
# initialize line matrix
xy = np.zeros((len(ps),2))

# direction variable (0-3)
der = 0


# loop over primes
for i in range(1,len(ps)):
    
    # compute distance
    dist = ps[i] - ps[i-1]
    
    # update the coordinates
    if der==0:
      xy[i,:] = [ xy[i-1,0], xy[i-1,1]+dist ]
    elif der==1:
      xy[i,:] = [ xy[i-1,0]+dist, xy[i-1,1] ]
    elif der==2:
      xy[i,:] = [ xy[i-1,0], xy[i-1,1]-dist ]
    elif der==3:
      xy[i,:] = [ xy[i-1,0]-dist, xy[i-1,1] ]
    
    # update direction
    der = (der+1)%4

# mean-center the image
xy -= np.mean(xy,axis=0)

In [ ]:
# static drawing
fig = plt.subplots(1,figsize=(8,8))
plt.plot(xy[:,0],xy[:,1],'k')
plt.gca().set_aspect(1./plt.gca().get_data_ratio())
plt.axis('off')
plt.show()

In [ ]:
# function to draw the plots
def aframe(i):
  ph.set_xdata(xy[:i,0])
  ph.set_ydata(xy[:i,1])
  return ph

# setup the image
fig,ax = plt.subplots(1,figsize=(8,8))
ph, = ax.plot(xy[:,0],xy[:,1],'k')
ax.set_aspect(1./ax.get_data_ratio())
ax.axis('off')

# and create the animation!
frameskip = 50
ani = animation.FuncAnimation(fig, aframe, range(0,len(xy),frameskip))
ani